In [7]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, lpSum, LpVariable, LpAffineExpression


In [8]:
# Create the model
model = LpProblem(name="small-problem", sense=LpMinimize)

In [9]:
p1 = LpVariable(name="p1", lowBound=0, upBound=1)
p21 = LpVariable(name="p21", lowBound=0, upBound=1)
p22 = LpVariable(name="p22", lowBound=0, upBound=1)
p31 = LpVariable(name="p31", lowBound=0, upBound=1)
p32 = LpVariable(name="p32", lowBound=0, upBound=1)
p33 = LpVariable(name="p33", lowBound=0, upBound=1)
p34 = LpVariable(name="p34", lowBound=0, upBound=1)
c = LpVariable(name="c")

In [10]:
con1 = (1-p1)+(1-p21)+(1-p31) <= c
con2 = (1-p1)+(1-p21)+(p31-1) <= c
con3 = (1-p1)+(p21-1)+(1-p32) <= c
con4 = (-p1)+(p21)+(p32) <= c
con5 = (p1-1)+(1-p22)+(1-p33) <= c
con6 = (p1)+(-p22)+(p33) <= c
con7 = (p1)+(p22)+(p34) <= c
con8 = (p1)+(p22)+(-p34) <= c


In [11]:
model += con1 
model += con2
model += con3  
model += con4
model += con5
model += con6
model += con7
model += con8


In [12]:
model += c

In [13]:
model

small-problem:
MINIMIZE
1*c + 0
SUBJECT TO
_C1: - c - p1 - p21 - p31 <= -3

_C2: - c - p1 - p21 + p31 <= -1

_C3: - c - p1 + p21 - p32 <= -1

_C4: - c - p1 + p21 + p32 <= 0

_C5: - c + p1 - p22 - p33 <= -1

_C6: - c + p1 - p22 + p33 <= 0

_C7: - c + p1 + p22 + p34 <= 0

_C8: - c + p1 + p22 - p34 <= 0

VARIABLES
c free Continuous
p1 <= 1 Continuous
p21 <= 1 Continuous
p22 <= 1 Continuous
p31 <= 1 Continuous
p32 <= 1 Continuous
p33 <= 1 Continuous
p34 <= 1 Continuous

In [14]:
status = model.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/hadar/Dev/RL/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_7/xf1wjsfx5bs9p2klwbsmtf640000gq/T/c9204e125db84271a98bb6e274cd7bd0-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_7/xf1wjsfx5bs9p2klwbsmtf640000gq/T/c9204e125db84271a98bb6e274cd7bd0-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 47 RHS
At line 56 BOUNDS
At line 65 ENDATA
Problem MODEL has 8 rows, 8 columns and 32 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 8 (0) rows, 8 (0) columns and 32 (0) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 0 ( 0%) - largest zero change 5.8145531e-05
0  Obj 0 Primal inf 5.9999996 (4) Dual inf 0.0099999 (1) w.o. free dual inf (0)
7  Obj 0.75004551
Optimal - objective value 0.75
Optimal objective 0.75 - 7 i

In [15]:
for var in model.variables():
    print(f"{var.name}: {var.value()}")

c: 0.75
p1: 0.5
p21: 0.75
p22: 0.25
p31: 1.0
p32: 0.5
p33: 0.5
p34: 0.0


In [53]:
class Node:
    def __init__(self, value, depth, parent=None):
        self.value = value
        self.depth = depth
        self.lp_variable = LpVariable(name=f"p{depth}_{value}", lowBound=0, upBound=1)
        self.left = None
        self.right = None
        self.parent = parent
    
    def get_left_child(self):
        return self.left
    
    def get_right_child(self):
        return self.right
    
    def get_parent(self):
        return self.parent
    
    def print_tree(self, level=0):
        """Recursively prints the tree structure."""
        print(' ' * 6 * level + f'- {self.depth}_{self.value}' + f'({self.lp_variable.value()})')
        if self.left:
            self.left.print_tree(level + 1)
        if self.right:
            self.right.print_tree(level + 1)
            

def generate_full_binary_tree(depth, current_depth=0, parent=None, value=1):
    """Recursively generates a full binary tree to the specified depth."""
    if depth < 0 or current_depth > depth:
        return None
    
    # Create the current node
    node = Node(value, current_depth, parent)
    
    # If not at the desired depth, create left and right children
    if current_depth < depth:
        node.left = generate_full_binary_tree(depth, current_depth + 1, node, value*2)
        node.right = generate_full_binary_tree(depth, current_depth + 1, node, value*2 + 1)
    
    return node


def get_lp_expressions(node, expression_0=None, expression_1=None):
    if not node.left and not node.right: # Leaf node
        return [expression_0, expression_1]
    
    expression_left_0 = expression_0 + (1 - node.lp_variable)
    expression_left_1 = expression_1 + (-node.lp_variable)
    expressions_left = get_lp_expressions(node.left, expression_left_0, expression_left_1)
    
    expression_right_0 = expression_0 + (node.lp_variable - 1)
    expression_right_1 = expression_1 + (node.lp_variable)
    expressions_right = get_lp_expressions(node.right, expression_right_0, expression_right_1)
    
    #return all the results as array with 1 dimension
    return expressions_left + expressions_right
    
    

def get_lp_constraints(node, c):
    constraints_dict = {}
    
    expressions = get_lp_expressions(node, LpAffineExpression(), LpAffineExpression())
    
    for expression in expressions:
        # Separate the LHS and RHS of the expression
        lhs_expression = expression - expression.constant
        rhs_value = -expression.constant  # Assuming expression is <= 0
        
        lhs_str = str(lhs_expression)  # Use LHS for comparison
        
        # Check if this LHS is already in the dictionary
        if lhs_str in constraints_dict:
            # Keep the most restrictive constraint (smaller RHS value)
            existing_rhs_value = -constraints_dict[lhs_str].constant
            if rhs_value < existing_rhs_value:
                constraints_dict[lhs_str] = lhs_expression - c <= rhs_value
        else:
            constraints_dict[lhs_str] = lhs_expression - c <= rhs_value
    
    # Return the list of unique, most restrictive constraints
    return list(constraints_dict.values())

    
# Example usage:|
T = 10 # Depth of the tree
root = generate_full_binary_tree(T)
c = LpVariable(name="c")
model2 = LpProblem(name="experts-problem-1", sense=LpMinimize)
constraints = get_lp_constraints(root, c) 
for constrains in constraints:
    print(constrains)


-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 - p7_128 - p8_256 - p9_512 <= -10
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 - p7_128 - p8_256 + p9_512 <= -8
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 - p7_128 + p8_256 - p9_513 <= -8
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 - p7_128 + p8_256 + p9_513 <= -6
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 + p7_128 - p8_257 - p9_514 <= -8
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 + p7_128 - p8_257 + p9_514 <= -6
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 + p7_128 + p8_257 - p9_515 <= -6
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 + p7_128 + p8_257 + p9_515 <= -4
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 + p6_64 - p7_129 - p8_258 - p9_516 <= -8
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 + p6_64 - p7_129 - p8_258 + p9_516 <= -6
-c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 + p6_64 - p7_129 + p8_258 - p9_517 <= -6
-c - p0_1 - p1_2 - p2_4 - p3_8 

In [54]:
model2 += c
for constraint in constraints:
    model2 += constraint

In [55]:
model2

experts-problem-1:
MINIMIZE
1*c + 0
SUBJECT TO
_C1: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 - p7_128 - p8_256
 - p9_512 <= -10

_C2: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 - p7_128 - p8_256
 + p9_512 <= -8

_C3: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 - p7_128 + p8_256
 - p9_513 <= -8

_C4: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 - p7_128 + p8_256
 + p9_513 <= -6

_C5: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 + p7_128 - p8_257
 - p9_514 <= -8

_C6: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 + p7_128 - p8_257
 + p9_514 <= -6

_C7: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 + p7_128 + p8_257
 - p9_515 <= -6

_C8: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 - p6_64 + p7_128 + p8_257
 + p9_515 <= -4

_C9: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 + p6_64 - p7_129 - p8_258
 - p9_516 <= -8

_C10: - c - p0_1 - p1_2 - p2_4 - p3_8 - p4_16 - p5_32 + p6_64 - p7_129
 - p8_258 + p9_51

In [56]:
model2.solve()
for var in model2.variables():
    print(f"{var.name}: {var.value()}")

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/hadar/Dev/RL/.venv/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/_7/xf1wjsfx5bs9p2klwbsmtf640000gq/T/faf21560635c49dca0361f890ddc8afd-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /var/folders/_7/xf1wjsfx5bs9p2klwbsmtf640000gq/T/faf21560635c49dca0361f890ddc8afd-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 1029 COLUMNS
At line 12295 RHS
At line 13320 BOUNDS
At line 14345 ENDATA
Problem MODEL has 1024 rows, 1024 columns and 11264 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 1024 (0) rows, 1024 (0) columns and 11264 (0) elements
Perturbing problem by 0.001% of 1 - largest nonzero change 0 ( 0%) - largest zero change 9.9892908e-05
0  Obj 0 Primal inf 1260 (386) Dual inf 0.0099999 (1) w.o. free dual inf (0)
95  Obj 0.64420211 Primal inf 870.49995 (454)
190  

In [58]:
print(c.value())

1.2304688


In [57]:
root.print_tree()

- 0_1(0.5)
      - 1_2(0.63671875)
            - 2_4(0.7734375)
                  - 3_8(0.890625)
                        - 4_16(0.96875)
                              - 5_32(1.0)
                                    - 6_64(1.0)
                                          - 7_128(1.0)
                                                - 8_256(1.0)
                                                      - 9_512(1.0)
                                                            - 10_1024(None)
                                                            - 10_1025(None)
                                                      - 9_513(1.0)
                                                            - 10_1026(None)
                                                            - 10_1027(None)
                                                - 8_257(1.0)
                                                      - 9_514(1.0)
                                                            - 10_1028(None)
              